### Práctica de José Luis Jerez

In [1]:
import spark.implicits._
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType, DoubleType, TimestampType}

// Carga de dataset en RDD - Spark Core
val data = sc.textFile("datasetPracticaFinal/TransaccionesNew.csv")

//Carga de dataset en DataFrame - SparkSQL
//val df = spark.sqlContext.read.format("csv")
//  .option("header", "true")
//  .option("inferSchema", "true")
//  .load("datasetPracticaFinal/TransaccionesNew.csv")

val customSchema = StructType(Array(
    // StructField("Transaction_date", TimestampType, true),
    StructField("Transaction_date", StringType, true),
    StructField("Product", StringType, true),
    StructField("Price", IntegerType, true),
    StructField("Payment_Type", StringType, true),
    StructField("Name", StringType, true),
    StructField("City", StringType, true),
    StructField("Account_Created", StringType, true),
    StructField("Last_Login", StringType, true),
    StructField("Latitude", DoubleType, true),
    StructField("Longitude", DoubleType, true),
    StructField("Description", StringType, true)))

val df = spark.sqlContext.read.format("csv")
  .option("header", "true")
  .schema(customSchema)
  .load("datasetPracticaFinal/TransaccionesNew.csv")

df.printSchema()

// Registra el dataframe en una tabla con el nombre pasado por parámetro
df.createOrReplaceTempView("transacciones")

Intitializing Scala interpreter ...

Spark Web UI available at http://4c887c19028a:4040
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1523832854347)
SparkSession available as 'spark'


root
 |-- Transaction_date: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Account_Created: string (nullable = true)
 |-- Last_Login: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Description: string (nullable = true)



import spark.implicits._
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType, DoubleType, TimestampType}
data: org.apache.spark.rdd.RDD[String] = datasetPracticaFinal/TransaccionesNew.csv MapPartitionsRDD[1] at textFile at <console>:30
customSchema: org.apache.spark.sql.types.StructType = StructType(StructField(Transaction_date,StringType,true), StructField(Product,StringType,true), StructField(Price,IntegerType,true), StructField(Payment_Type,StringType,true), StructField(Name,StringType,true), StructField(City,StringType,true), StructField(Account_Created,StringType,true), StructField(Last_Login,StringType,true), StructField(Latitude,DoubleType,true), StructField(Longitude,DoubleType,true), StructField(Description,...

- Tarea 1: 
  + Definición de objeto transacción
    + DNI (Este dato no viene en el dataset, se generará con un acumulador de forma incremental (1 a N)
    + Tarjeta de crédito
    + Importe
    + Descripción del pago
    + Ciudad
    + Categoría
  + Definición objeto cliente:DNI
    + Nombre
    + Número de cuenta
    + Balance
    + Tarjeta de crédito
    + Ciudad

Es necesario definir previamente el objeto "geolocalizacion"
- Definición de objeto geolocalizacion
    + Latitud
    + Longitud
    + Ciudad
    + Pais

In [2]:
case class Geolocalizacion(latitud: Double, longitud: Double, ciudad: String, pais: String)

defined class Geolocalizacion


In [3]:
case class Transaccion(DNI: Long, longitud: Double, categoria: String, tarjetaCredito: String, geolocalizacion: Geolocalizacion)
case class Cliente(DNI: Long, nombre: String, cuenaCorriente: String)

defined class Transaccion
defined class Cliente


Implementa las siguientes categorizaciones usando funciones Core de Spark.
Los resultados se almacenarán combinando diferentes formatos de ficheros (CSV, Avro, Parquet etc)

- Tarea 2: Agrupa todos los clientes por ciudad.El objetivo sería contar todos las transacciones ocurridas por ciudad.


In [5]:
// Utilizando RDDs

val words = data.map(_.split(',')(5)).filter( _ != "ciudad" )
val tarea2 = words.countByValue
sc.parallelize(tarea2.toSeq).saveAsTextFile("tarea2")

words: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[10] at filter at <console>:34
tarea2: scala.collection.Map[String,Long] = Map(Drogheda -> 1, Saint Catharines -> 1, Bunbury -> 1, Tel Aviv -> 1, Amsterdam -> 3, Ehingen an der Donau -> 1, Yellowknife -> 1, Navan -> 1, "Spring Lake Park            " -> 1, Danderyd -> 1, Lucca -> 1, "Atlanta                     " -> 2, Killiney -> 1, "Stamford                    " -> 1, "Roanoke                     " -> 1, "Chula Vista                 " -> 1, "Larchmont                   " -> 1, "New Rochelle                " -> 2, Stocksund -> 1, Scamander -> 1, "Atchison                    " -> 1, "Temecula                    " -> 1, Oxford -> 3, Calne -> 1, Telgte -> 1, "Swampscott                  " -> 1, Waterford -> 2, "Pacific Palisades    ...

- Tarea 3: Encuentra aquellos clientes que hayan realizado pagos superiores a 500

In [6]:
// Utilizando RDDs

val c = data.map(line => {
  val lines = line.split(',')
  (lines(2), lines(4))
})
c.first.getClass
val cabecera = c.first()
val rddSinCabecera = c.filter(row => !row.equals(cabecera))
// val rddSinCabecera2 = rddSinCabecera.map(fila => (fila._1.toInt,fila._2))
val tarea3 = rddSinCabecera.filter(fila => fila._1.toInt > 500)
tarea3.toDF.write.csv("tarea3")

c: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[16] at map at <console>:34
cabecera: (String, String) = (Price,Name)
rddSinCabecera: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[17] at filter at <console>:40
tarea3: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[18] at filter at <console>:42


In [7]:
// Utilizando DataFrames

val tarea3_Spark_sql = spark.sql("SELECT Name, Price FROM transacciones WHERE Price > 500")
tarea3_Spark_sql.show
tarea3.toDF.write.parquet("tarea3_Spark_sql")

+-----------------+-----+
|             Name|Price|
+-----------------+-----+
|         carolina| 1200|
|           Betina| 1200|
|Federica e Andrea| 1200|
|            Gouya| 1200|
|          Gerd W | 3600|
|         LAURENCE| 1200|
|            Fleur| 1200|
|             adam| 1200|
|  Renee Elisabeth| 1200|
|            Aidan| 1200|
|            Stacy| 1200|
|            Heidi| 1200|
|            Sean | 1200|
|          Georgia| 1200|
|          Richard| 1200|
|           Leanne| 1200|
|            Janet| 1200|
|          barbara| 1200|
|           Sabine| 3600|
|             Hani| 1200|
+-----------------+-----+
only showing top 20 rows



tarea3_Spark_sql: org.apache.spark.sql.DataFrame = [Name: string, Price: int]


- Tarea 4: Obtén todas las transacciones agrupadas por cliente cuya la la ciudad X.

In [8]:
val tarea4 = spark.sql("SELECT City, Name FROM transacciones WHERE City = 'Amsterdam' ORDER BY Name")
tarea4.show
//tarea4.toDF.write.csv("tarea4")
tarea4.toDF.write.format("com.databricks.spark.avro").save("tarea4")

+---------+---------+
|     City|     Name|
+---------+---------+
|Amsterdam|Antonella|
|Amsterdam| Caterina|
|Amsterdam|    sunny|
+---------+---------+



org.apache.spark.sql.AnalysisException:  Failed to find data source: com.databricks.spark.avro. Please find an Avro package at http://spark.apache.org/third-party-projects.html;

- Tarea 5: Filtra todas las operaciones cuya categoría sea Ocio (Restaurant, Cinema y Sports)

In [9]:
val tarea5 = spark.sql("SELECT * FROM transacciones Where Description = 'Restaurant' OR Description = 'Cinema' OR Description = 'Sports'")
tarea5.show
tarea5.toDF.write.json("tarea5")

+----------------+--------+-----+------------+-----------------+--------------------+---------------+-------------+-----------+----------+-----------+
|Transaction_date| Product|Price|Payment_Type|             Name|                City|Account_Created|   Last_Login|   Latitude| Longitude|Description|
+----------------+--------+-----+------------+-----------------+--------------------+---------------+-------------+-----------+----------+-----------+
|     1/2/09 4:53|Product1| 1200|        Visa|           Betina|Parkville        ...|            124|  1/2/09 7:49|     39.195| -94.68194| Restaurant|
|    1/2/09 13:08|Product1| 1200|  Mastercard|Federica e Andrea|Astoria          ...|            125| 1/3/09 12:32|   46.18806|   -123.83|     Cinema|
|    1/3/09 14:44|Product1| 1200|        Visa|            Gouya|              Echuca|            126| 1/3/09 14:22|-36.1333333|    144.75|     Sports|
|    1/8/09 16:24|Product1| 1200|        Visa|         jennifer|Phoenix          ...|         

tarea5: org.apache.spark.sql.DataFrame = [Transaction_date: string, Product: string ... 9 more fields]


- Tarea 6: Obtén las últimas transacciones de cada cliente en los últimos 30 días

In [10]:
import org.apache.spark.sql.functions.unix_timestamp
val df2 = df.toDF("Transaction_date","Product","Price","Payment_Type","Name","City","Account_Created","Last_Login","Latitude","Longitude","Description")
val td = unix_timestamp($"Transaction_date", "MM/dd/yy HH:mm").cast("timestamp")
val tarea6 = df2.withColumn("td", td).select($"td", $"Transaction_date", $"Name", $"City")
   .filter($"td" > "2009-01-15 00:00:00")
   .orderBy($"td".asc)
tarea6.show
tarea6.write.csv("tarea6")

+-------------------+----------------+------------+--------------------+
|                 td|Transaction_date|        Name|                City|
+-------------------+----------------+------------+--------------------+
|2009-01-15 01:41:00|    1/15/09 1:41|       Karen|           Roquefort|
|2009-01-15 02:40:00|    1/15/09 2:40|      Alexis|Genoa            ...|
|2009-01-15 03:43:00|    1/15/09 3:43|   Stephanie|               Rouen|
|2009-01-15 04:12:00|    1/15/09 4:12|   catherine|Keller           ...|
|2009-01-15 05:11:00|    1/15/09 5:11|         Pam|              London|
|2009-01-15 05:27:00|    1/15/09 5:27|     Derrick|           North Bay|
|2009-01-15 05:47:00|    1/15/09 5:47|Dave and Amy|Norfolk          ...|
|2009-01-15 07:26:00|    1/15/09 7:26|      Celina|            Den Haag|
|2009-01-15 08:07:00|    1/15/09 8:07|     Kirstie|Royal Oak        ...|
|2009-01-15 08:10:00|    1/15/09 8:10|      Scott |Royal Oak        ...|
|2009-01-15 08:12:00|    1/15/09 8:12|   Katherine|

import org.apache.spark.sql.functions.unix_timestamp
df2: org.apache.spark.sql.DataFrame = [Transaction_date: string, Product: string ... 9 more fields]
td: org.apache.spark.sql.Column = CAST(unix_timestamp(Transaction_date, MM/dd/yy HH:mm) AS TIMESTAMP)
tarea6: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [td: timestamp, Transaction_date: string ... 2 more fields]


- Tarea 7[Opcional]: Encuentra por cada transacción a través de su longitud y latitud cual es el país a donde pertenece la transacción y guardalo en el campo país.

In [ ]:
@throws(classOf[java.io.IOException])
@throws(classOf[java.net.SocketTimeoutException])
def get(url: String,
        connectTimeout: Int = 5000,
        readTimeout: Int = 5000,
        requestMethod: String = "GET") =
{
    import java.net.{URL, HttpURLConnection}
    val connection = (new URL(url)).openConnection.asInstanceOf[HttpURLConnection]
    connection.setConnectTimeout(connectTimeout)
    connection.setReadTimeout(readTimeout)
    connection.setRequestMethod(requestMethod)
    val inputStream = connection.getInputStream
    val content = io.Source.fromInputStream(inputStream).mkString
    if (inputStream != null) inputStream.close
    content
}

In [11]:
object GetRestContent extends App {

  val url = "http://api.hostip.info/get_json.php?ip=12.215.42.19"
  val result = scala.io.Source.fromURL(url).mkString
  println(result)

}

defined object GetRestContent


In [ ]:
GetRestContent.url